In [1103]:
import numpy as np 
from scipy import sparse 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

np.random.seed(10)

In [1126]:
# randomly generate data 
N = 10 # number of training sample 
d = 2 # data dimension 
C = 3 # number of classes 

X = np.random.randn(N, d)
y = np.random.randint(1, 4, (N,1))
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y)
Y = enc.transform(y).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [1131]:
def softmax(Z):
    return (np.exp(Z).T/np.sum(np.exp(Z), axis=1)).T # (dim(C,N)/dim(N)).T = (N,C)

def cost(xi, yi, w):
    ai = softmax(xi.dot(w.T))
    return -np.sum(yi*np.log(ai))

def SGD(xi, yi, w):
    ai = softmax(xi.dot(w.T))
    ei = ai - yi
    return ei.T.dot(xi)

def check_gradient(X, Y, cost, differential):

    def numerical_gradient(x, y, w, cost):
        eps = 1e-6
        g = np.zeros_like(w)
        for i in range(w.shape[0]):
            for j in range(w.shape[1]):
                w_p = w.copy()
                w_n = w.copy()
                w_p[i, j] += eps 
                w_n[i, j] -= eps
                g[i,j] = (cost(x, y, w_p) - cost(x, y, w_n))/(2*eps)
        return g 

    W = np.random.randn(Y.shape[1], X.shape[1])
    print('Coefficients size:', W.shape)
    grad1 = differential(X, Y, W)
    grad2 = numerical_gradient(X, Y, W, cost)
    return True if np.linalg.norm(grad1 - grad2) < 1e-6 else False 

print('Checking gradient formula...', check_gradient(X, Y, cost, SGD))

Coefficients size: (3, 2)
Checking gradient formula... True


In [1127]:
def softmax_regression(X, y, eta = 1, tol = 1e-4, early_stopping = True, epochs = 50000):
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoder.fit(y)
    Y = encoder.transform(y).toarray()

    W_init = np.random.randn(Y.shape[1], X.shape[1])
    W = [W_init]    
    it = 0
    N = X.shape[0]
    d = X.shape[1]
    
    check_w_after = 10
    for epoch in range(1,epochs):
        # shuffle data 
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[i,:].reshape(1, d)   # dim(1,d)
            yi = Y[i,:].reshape(1, C)   # dim(1,C)
            ai = softmax(xi.dot(W[-1].T))   # dim(1,C)
            W_new = W[-1] + eta*np.dot((yi - ai).T, xi) # dim(C,d)
            # stopping criteria
            if epoch % check_w_after == 0 and early_stopping == True:                
                if np.linalg.norm(W_new - W[-check_w_after]) < tol:
                    return W[-1], epoch
            W.append(W_new)
    return W[-1], epoch


W, epoch = softmax_regression(X_train, y_train, 0.5, 1e-6)
print('Coefficients found by built-in model:')
W

Coefficients found by built-in model:


array([[-18.89840473,   9.13241615],
       [  7.14007442,  -5.14470842],
       [  9.87068953,  -4.33744232]])

In [813]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [1128]:
model = LogisticRegression(C = 1e5, solver = 'lbfgs', multi_class = 'multinomial', random_state = 0)
model.fit(X, y.reshape(X.shape[0]))
print('Coefficients found by Sklearn model:')
model.coef_

Coefficients found by Sklearn model:


array([[-12.76581404,  19.72232837],
       [-11.9852142 , -43.6974649 ],
       [ 24.75102825,  23.97513653]])

### Evaluation

In [1129]:
# Evaluate models:
y_pred = model.predict(X_test)
print(f'Accuracy score: {accuracy_score(y_test, y_pred)*100:.2f} %')
y_pred, y_test.T

Accuracy score: 100.00 %


(array([3, 1, 3]), array([[3, 1, 3]]))

In [1130]:
model.coef_ = W
y_pred = model.predict(X_test)
print(f'Accuracy score: {accuracy_score(y_test, y_pred)*100:.2f} %')
y_pred, y_test.T

Accuracy score: 100.00 %


(array([3, 1, 3]), array([[3, 1, 3]]))